In [1]:
import pandas as pd
import numpy as np
from tqdm import notebook
import ast
import re
from fuzzywuzzy import fuzz

In [15]:
notebook.tqdm.pandas()
clean = pd.read_csv('../clean_data.csv')
voc = pd.read_csv('../vocclustered.csv', sep='	')

In [78]:
name_df.head(20)

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,clean_namen,name
0,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,NaN,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,"[Trijntje Lansink, Martinus Ciprianus, Simon v...",Trijntje Lansink
1,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,NaN,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,"[Trijntje Lansink, Martinus Ciprianus, Simon v...",Martinus Ciprianus
2,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,NaN,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,"[Trijntje Lansink, Martinus Ciprianus, Simon v...",Simon van Zuijderwijk
3,17599c0c-3305-165c-aae3-eddbb497e4b4,358,JAN VERLEIJ,11885,107542,Testament,1741-03-16,nederlands,NaN,"[{'voornaam': 'Trijntje', 'tussenvoegsel': Non...","['KLAB05372000012.JPG', 'KLAB05372000013.JPG',...",N: 4 1e de Testateuren hebben verklaerd te Sam...,"[Trijntje Lansink, Martinus Ciprianus, Simon v...",Margareta van Zuijdewijk
4,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,"[Gerard Plaatman, Anna Johanna Romano, Antje P...",Gerard Plaatman
5,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,"[Gerard Plaatman, Anna Johanna Romano, Antje P...",Anna Johanna Romano
6,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,"[Gerard Plaatman, Anna Johanna Romano, Antje P...",Antje Pieters Blok
7,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,"[Gerard Plaatman, Anna Johanna Romano, Antje P...",Cornelis van Ommeren
8,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,"[Gerard Plaatman, Anna Johanna Romano, Antje P...",Jan Carel Romano
9,1b037028-bea5-0fa8-e6c5-68155f5f2b21,358,JAN VERLEIJ,11885,108046,Attestatie,1741-07-26,nederlands,\nordentelijke vrijage\n,"[{'voornaam': 'Gerard', 'tussenvoegsel': None,...","['KLAB05372000193.JPG', 'KLAB05372000194.JPG',...",Verklaing gepasseert den 26 Julij 1741No: 60- ...,"[Gerard Plaatman, Anna Johanna Romano, Antje P...",Maurina Romano


In [66]:
uuid = []
name = []
for y, z in notebook.tqdm(clean.iterrows()):
    for x in ast.literal_eval(z.namen):
        if x['tussenvoegsel'] != None:
            name.append(x['voornaam'] + " " + x['tussenvoegsel'] + " " + x['achternaam'])
            uuid.append(z.uuid)
        elif x['voornaam'] and x['achternaam'] != None:
            name.append(x['voornaam'] + " " + x['achternaam'])
            uuid.append(z.uuid)
name_list = pd.DataFrame(data={'uuid':uuid, 'name':name}, columns=['uuid', 'name'])

In [69]:
name_df = clean.merge(name_list)

In [96]:
def fuzzy_search(name, distance):
    names = np.where((voc.fullNameNormalized.apply(fuzz.ratio, args=[name]) >= 90) | 
                     (voc.fullNameOriginal.dropna().apply(fuzz.ratio, args=[name]) >= 90))
    final = (name, names)
    return final

def find_matches(names, distance):
    name_list = {}
    final = []
    for x in notebook.tqdm(names):
        if x in name_list:
            final.append((x, name_list[x]))
        else:
            result = fuzzy_search(x, distance)
            name_list[x] = result
            final.append((x, result))
    return final

In [97]:
test = find_matches(name_df[0:100].name, 90)

In [49]:
1:41 apply

nan

In [98]:
test

[('Trijntje Lansink', ('Trijntje Lansink', (array([], dtype=int64),))),
 ('Martinus Ciprianus', ('Martinus Ciprianus', (array([], dtype=int64),))),
 ('Simon van Zuijderwijk',
  ('Simon van Zuijderwijk', (array([], dtype=int64),))),
 ('Margareta van Zuijdewijk',
  ('Margareta van Zuijdewijk', (array([], dtype=int64),))),
 ('Gerard Plaatman', ('Gerard Plaatman', (array([], dtype=int64),))),
 ('Anna Johanna Romano', ('Anna Johanna Romano', (array([], dtype=int64),))),
 ('Antje Pieters Blok', ('Antje Pieters Blok', (array([], dtype=int64),))),
 ('Cornelis van Ommeren',
  ('Cornelis van Ommeren',
   (array([288686, 310196, 321893, 397388, 401215, 404902, 467060, 634007,
           638370, 639123, 646721], dtype=int64),))),
 ('Jan Carel Romano', ('Jan Carel Romano', (array([], dtype=int64),))),
 ('Maurina Romano', ('Maurina Romano', (array([], dtype=int64),))),
 ('Trijntje Baptist', ('Trijntje Baptist', (array([], dtype=int64),))),
 ('Dirk van den Horst',
  ('Dirk van den Horst',
   (array([

In [91]:
np.where(name_df.name.value_counts() == 1)[0].shape

(28642,)

In [ ]:
28642 + 9000